In [33]:
import scipy.io
import pandas as pd
import numpy as np
from itertools import chain
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot

In [34]:
def anova_2f(data):
    
    '''
    input should be a pandas.DataFrame with arrays y, f1 and f2 and index.
    y = n spikes fired
    f1 = success of memory encoding
    f2 = stimulus identity
    output will be the anova table in the form of a pandas.DataFrame with columns
    'encoding', 'stimID', 'encoding:stimID' (=interaction) and 'Residual'
    and e.g. p values can be accessed through aov['p']
    based on
    http://www.pybloggers.com/2016/03/three-ways-to-do-a-two-way-anova-with-python/
    '''
    
    # degrees of freedom
    N = len(data.y)
    df_f1 = len(data.f1.unique())-1
    df_f2 = len(data.f2.unique())-1
    df_f1xf2 = df_f1*df_f2
    df_w = N - (len(data.f1.unique())*len(data.f2.unique()))

    # sum of squares
    grand_mean = data.y.mean()
    ssq_f1 = sum([(data[data.f1 == i].y.mean()-grand_mean)**2 for i in data.f1])
    ssq_f2 = sum([(data[data.f2 == i].y.mean()-grand_mean)**2 for i in data.f2])
    ssq_t  = sum((data.y - grand_mean)**2)

    # ssq_w (sum of squares within) how far is each of the data points away from the 
    # mean of its particular group? - compute ssq_w for 2 groups
    memF = data[data.f1 == 0]
    memC = data[data.f1 == 1]
    # create vector with individual group means
    memF_mean_per_stim = [memF[memF.f2 == s].y.mean() for s in memF.f2]
    memC_mean_per_stim = [memC[memC.f2 == s].y.mean() for s in memC.f2]
    ssq_w = sum((memC.y - memC_mean_per_stim)**2) + sum((memF.y - memF_mean_per_stim)**2)

    # since we have a 2 way design we need to calculate the sum of sqares for the 
    # interactoin of factor 1 and factor 2
    ssq_f1xf2 = ssq_t - ssq_f1 - ssq_f2 - ssq_w

    # calculate the mean square for each factor, interaction & within
    ms_f1    = ssq_f1    / df_f1    # mean square f1
    ms_f2    = ssq_f2    / df_f2    # mean square f2
    ms_f1xf2 = ssq_f1xf2 / df_f1xf2 # mean square f1xf2 
    ms_w     = ssq_w     / df_w

    # F-ratio
    f_f1    = ms_f1    / ms_w
    f_f2    = ms_f2    / ms_w
    f_f1xf2 = ms_f1xf2 / ms_w

    # p-values
    p_f1    = stats.f.sf(f_f1,    df_f1,    df_w)
    p_f2    = stats.f.sf(f_f2,    df_f2,    df_w)
    p_f1xf2 = stats.f.sf(f_f1xf2, df_f1xf2, df_w)

    results = {'sum_sq': [ssq_f1, ssq_f2, ssq_f1xf2, ssq_w],
                   'df': [ df_f1,  df_f2,  df_f1xf2,  df_w],
                    'F': [  f_f1,   f_f2,   f_f1xf2,  'NaN'],
                    'p': [  p_f1,   p_f2,   p_f1xf2,  'NaN']}
    columns = ['sum_sq', 'df', 'F', 'p']

    aov_table = pd.DataFrame(results, columns=columns, index = 
                             ['encoding', 'stimID', 'encoding:stimID', 'Residual'])

    # add effect size, measures eta squared and omega squared (less biased)
    def eta_squared(aov):
        aov['eta_sq'] = 'NaN'
        aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
        return aov

    def omega_squared(aov):
        mse = aov['sum_sq'][-1]/aov['df'][-1]
        aov['omega_sq'] = 'NaN'
        aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*mse))/(sum(aov['sum_sq'])+mse)

    eta_squared(aov_table)
    omega_squared(aov_table)

    return aov_table

In [35]:
# load data
f_a = scipy.io.loadmat('./anova_short.mat')
n_units = f_a['anova_cell'].shape[0]
infomat = f_a['infocell']
p_vals = np.zeros((n_units, 7))
p_vals[:,:4] = infomat[:,:4].astype(int)

# loop through all units 
for unit in range(n_units):
    frame_dict = {'y': np.array(list(chain.from_iterable(f_a['anova_cell'][unit][0]))), # firing rate 
                 'f1': np.array(list(chain.from_iterable(f_a['anova_cell'][unit][1]))), # mem
                 'f2': np.array(list(chain.from_iterable(f_a['anova_cell'][unit][2])))} # stimulus id
    data = pd.DataFrame(data=frame_dict, index = range(len(frame_dict['y'])))
    aov_table = anova_2f(data)
    p_vals[unit, 4] = aov_table['p'][0] # success of encoding
    p_vals[unit, 5] = aov_table['p'][1] # simulus ID
    p_vals[unit, 6] = aov_table['p'][2] # interaction
    
#print('{0.2f}'.format(p_vals))
print("%.2f", p_vals)

%.2f [[  4.20000000e+01   2.00000000e+00   1.00000000e+00   1.00000000e+00
    5.22440002e-02   4.20911747e-02   1.00000000e+00]
 [  4.20000000e+01   2.00000000e+00   1.00000000e+00   2.00000000e+00
    9.58583659e-01   7.10213558e-01   5.90886560e-01]
 [  4.20000000e+01   2.00000000e+00   1.00000000e+00   3.00000000e+00
    5.61931081e-01   8.82510206e-01   2.36726688e-01]
 [  4.20000000e+01   2.00000000e+00   2.00000000e+00   1.00000000e+00
    3.46489278e-01   8.03714701e-01   2.81507358e-01]
 [  4.20000000e+01   2.00000000e+00   2.00000000e+00   2.00000000e+00
    6.46776422e-01   8.66085845e-01   3.01467948e-01]
 [  4.20000000e+01   2.00000000e+00   2.00000000e+00   3.00000000e+00
    3.96223534e-01   4.51619436e-01   4.23416866e-01]
 [  4.20000000e+01   2.00000000e+00   3.00000000e+00   1.00000000e+00
    2.05878290e-01   7.42267432e-01   3.09587891e-01]
 [  4.20000000e+01   2.00000000e+00   3.00000000e+00   2.00000000e+00
    1.79991138e-02   3.03215512e-01   7.46987237e-01]
 [ 